### 191_AED


* [issue 191](https://github.com/salgo60/Stockholm_Archipelago_Trail/issues/191)
* [this notebook](https://github.com/salgo60/Stockholm_Archipelago_Trail/tree/main/notebook/191_AED.ipynb)

In [1]:
import time
import datetime  
start_time = time.time()
start_str = datetime.datetime.now().strftime("%Y-%m-%d %H:%M")
print(f"Started: {start_str}")


Started: 2025-09-29 07:21


In [2]:
#!/usr/bin/env python3
"""
Generisk OSM↔Wikidata-koll via valfri SPARQL

Syfte
------
Skicka in *vilken SPARQL-fråga som helst* som returnerar ett ?item (Wikidata-objekt)
och **någon** form av OSM-id (node/way/relation) – t.ex. via standardegenskaperna
P10689 (node), P11693 (way), P402 (relation) eller som färdiga URL:er – så kontrollerar
skriptet att **motsvarande OSM-objekt pekar tillbaka** till Wikidata via `wikidata=Q…`.

Vad gör skriptet?
-----------------
1) Kör din SPARQL, tolkar resultatet och extraherar:
   - QID (från ?item)
   - OSM-ID:n (node/way/relation) från valfria fält/format.
2) Hämtar nämnda OSM-objekt direkt via Overpass (per id, ej bbox).
3) Validerar back-link:
   - ✅ **OK**: OSM har `wikidata`-tagg som *innehåller* samma QID (hanterar även semikolon-separerade listor).
   - ⚠️ **SAKNAS**: OSM saknar `wikidata`-tag helt.
   - ❌ **FEL**: OSM har `wikidata`, men den matchar **inte** QID.
4) Skriver ut **listor med klickbara länkar** för varje kategori och exporterar CSV.

Exempelanrop
------------
- `run_generic(SPARQL_QUERY)`

Notera
------
- Du kan fortfarande använda den tidigare `run()` för den hårdkodade SAT-frågan om du vill.
"""
from __future__ import annotations
import re
import math
import json
from typing import Dict, Iterable, List, Tuple

import pandas as pd
import requests

WIKIDATA_SPARQL = "https://query.wikidata.org/sparql"
OVERPASS_API = "https://overpass-api.de/api/interpreter"
USER_AGENT = "SAT-OSM-WD-Consistency/2.0 (contact: your-email@example.com)"

# -------------------------
# Hjälpfunktioner
# -------------------------

def make_headers() -> Dict[str, str]:
    return {"User-Agent": USER_AGENT}


def to_osm_prefixed_id(typ: str, osm_id: int) -> str:
    return {"node": "n", "way": "w", "relation": "r"}[typ] + str(int(osm_id))


def parse_osm_from_value(val: str) -> Tuple[str, int] | None:
    """Försök tolka ett OSM-id (typ, id) från godtycklig sträng/URL/tal.
    Stödjer t.ex. 12345, "12345", "https://www.openstreetmap.org/node/12345".
    Returnerar (typ, id) eller None.
    """
    if val is None:
        return None
    s = str(val).strip()

    # URL-format
    m = re.search(r"openstreetmap\.org\/(node|way|relation)\/(\d+)", s)
    if m:
        return m.group(1), int(m.group(2))

    # Rena tal: lämna typ okänd
    if s.isdigit():
        # Typ avgörs av fältnamnet – hanteras av anroparen
        return ("unknown", int(s))

    return None


# -------------------------
# Wikidata
# -------------------------

def fetch_wikidata_generic(sparql: str) -> pd.DataFrame:
    """Kör godtycklig SPARQL och extraherar QID och OSM-id:n ur valfria kolumner.
    Krav: ?item måste finnas och vara ett WD-objekt.
    Heuristik för OSM:
      - Leta kolumnnamn som innehåller något av: node|way|rel|osm|OSM
      - För varje sådan kolumn: tolka värdet som URL eller siffra.
      - Om typ är "unknown" – gissa typ utifrån kolumnnamn (node/way/relation).
    Returnerar DF med kolumner: qid, label (om finns), osm_type, osm_id
    (en rad per QID×OSM-referens). Om ingen OSM-referens hittas för en QID
    returneras ändå en rad utan OSM-typ/id för spårbarhet.
    """
    r = requests.get(WIKIDATA_SPARQL, params={"query": sparql, "format": "json"}, headers=make_headers())
    r.raise_for_status()
    rows = r.json()["results"]["bindings"]

    out_rows: List[Dict] = []
    for b in rows:
        item_uri = b.get("item", {}).get("value")
        if not item_uri:
            continue
        qid = item_uri.rsplit("/", 1)[-1]
        label = b.get("itemLabel", {}).get("value")

        # samla alla kandidater till OSM-kolumner
        osm_keys = [k for k in b.keys() if re.search(r"(node|way|rel|osm)", k, re.I) and k != "item"]

        found_any = False
        for k in osm_keys:
            raw = b[k].get("value")
            parsed = parse_osm_from_value(raw)
            if parsed:
                typ, oid = parsed
                if typ == "unknown":
                    # gissa från kolumnnamn
                    lk = k.lower()
                    if "node" in lk:
                        typ = "node"
                    elif "way" in lk:
                        typ = "way"
                    elif "rel" in lk:
                        typ = "relation"
                    else:
                        # kan ej avgöra – hoppa
                        continue
                out_rows.append({
                    "qid": qid, "label": label, "osm_type": typ, "osm_id": int(oid),
                    "osm_url": f"https://www.openstreetmap.org/{typ}/{int(oid)}",
                })
                found_any = True

        if not found_any:
            # lägg en rad utan OSM för spårbarhet
            out_rows.append({"qid": qid, "label": label, "osm_type": None, "osm_id": None, "osm_url": None})

    df = pd.DataFrame(out_rows).drop_duplicates()
    return df


# -------------------------
# Overpass
# -------------------------

def fetch_osm_by_ids(df_refs: pd.DataFrame) -> pd.DataFrame:
    """Hämta OSM-objekt per id för de rader som har osm_type/osm_id.
    Returnerar kolumner: osm_type, osm_id, url, wikidata_tag
    """
    subset = df_refs.dropna(subset=["osm_type", "osm_id"]).copy()
    if subset.empty:
        return pd.DataFrame(columns=["osm_type","osm_id","url","wikidata_tag"])   

    ids_by_type: Dict[str, List[int]] = {"node": [], "way": [], "relation": []}
    for _, r in subset.iterrows():
        t = r["osm_type"]
        if t in ids_by_type:
            ids_by_type[t].append(int(r["osm_id"]))

    def batched(lst: List[int], n: int = 200) -> Iterable[List[int]]:
        for i in range(0, len(lst), n):
            yield lst[i:i+n]

    rows: List[Dict] = []
    for t in ("node","way","relation"):
        if not ids_by_type[t]:
            continue
        for batch in batched(sorted(set(ids_by_type[t]))):
            ids_str = ",".join(str(i) for i in batch)
            q = f"""
            [out:json][timeout:50];
            {t}(id:{ids_str});
            out ids tags center;
            """.strip()
            rr = requests.post(OVERPASS_API, data={"data": q}, headers=make_headers())
            rr.raise_for_status()
            for el in rr.json().get("elements", []):
                tags = el.get("tags", {})
                rows.append({
                    "osm_type": t,
                    "osm_id": el.get("id"),
                    "url": f"https://www.openstreetmap.org/{t}/{el.get('id')}",
                    "wikidata_tag": tags.get("wikidata"),
                    "name": tags.get("name"),
                })

    return pd.DataFrame(rows)


# -------------------------
# Validering & utskrift
# -------------------------

def normalize_wikidata_values(val: str | None) -> List[str]:
    if not val:
        return []
    # Dela på semikolon/komma och trimma, ta bara sådant som ser ut som Q…
    parts = re.split(r"[;|,]", val)
    return [p.strip() for p in parts if re.match(r"^Q\d+$", p.strip(), re.I)]


def verify_backlinks(df_refs: pd.DataFrame, df_osm: pd.DataFrame) -> Dict[str, pd.DataFrame]:
    # join på (osm_type, osm_id)
    key = ["osm_type","osm_id"]
    merged = pd.merge(df_refs.dropna(subset=key), df_osm, on=key, how="left", suffixes=("","_osm"))

    ok_rows = []
    missing_rows = []
    wrong_rows = []

    for _, r in merged.iterrows():
        qid = r["qid"]
        tag = r.get("wikidata_tag")
        if pd.isna(tag) or not tag:
            missing_rows.append(r)
        else:
            vals = normalize_wikidata_values(tag)
            if qid in vals:
                ok_rows.append(r)
            else:
                wrong_rows.append(r)

    return {
        "ok": pd.DataFrame(ok_rows),
        "missing": pd.DataFrame(missing_rows),
        "wrong": pd.DataFrame(wrong_rows),
        "all_refs": df_refs,
        "all_osm": df_osm,
    }


def print_link_lists(res: Dict[str, pd.DataFrame]):
    def print_group(title: str, df: pd.DataFrame):
        print(f"\n{title}")
        print("-" * len(title))
        if df.empty:
            print("(inget)")
            return
        # Grupp: per QID
        for qid, grp in df.groupby("qid"):
            print(f"{qid}:")
            for _, r in grp.iterrows():
                print(f"  {r['url']}")

    print_group("❌ OSM har wikidata men pekar fel", res["wrong"])        
    print_group("✅ OSM back-link OK (wikidata matchar QID)", res["ok"])    
    print_group("⚠️ OSM saknar wikidata-tag (lägg till QID)", res["missing"]) 


# -------------------------
# Publika körfunktioner
# -------------------------

def run_generic(sparql_query: str, export_prefix: str = "wd_osm_backlinks"):
    print("Kör SPARQL…")
    refs = fetch_wikidata_generic(sparql_query)
    print(f"Rader från SPARQL (QID×OSM-ref): {len(refs)}")

    print("Hämtar OSM per id…")
    osm = fetch_osm_by_ids(refs)
    print(f"OSM-objekt hämtade: {len(osm)}")

    res = verify_backlinks(refs, osm)

    # Exportera CSV
    for name, df in res.items():
        path = f"{export_prefix}_{name}.csv"
        df.to_csv(path, index=False)
        print(f"↳ sparade {path} ({len(df)} rader)")

    # Listrapport med klickbara länkar
    print_link_lists(res)



def run_sat_example():
    run_generic(SAT_SPARQL, export_prefix="sat_backlinks")


In [3]:
# https://w.wiki/FT6j
run_generic("""
SELECT  ?item  ?itemLabel  ?coord ?img ?OSMnode ?OSMway ?OSMrel WHERE {
  ?item wdt:P31 wd:Q1450682;        # AED kopplade till SAT i WD
       wdt:P6104 wd:Q134294510
    OPTIONAL  { ?item wdt:P18 ?img. } # Bild
    OPTIONAL { ?item wdt:P625 ?coord }
    OPTIONAL { ?item wdt:P10689 ?OSMway }
    OPTIONAL { ?item wdt:P11693 ?OSMnode }
    OPTIONAL { ?item wdt:P402   ?OSMrel }

    SERVICE wikibase:label { bd:serviceParam wikibase:language "sv,en". }

}
ORDER BY ?itemlabel
""", export_prefix="sat_backlinks")


Kör SPARQL…
Rader från SPARQL (QID×OSM-ref): 26
Hämtar OSM per id…
OSM-objekt hämtade: 26
↳ sparade sat_backlinks_ok.csv (26 rader)
↳ sparade sat_backlinks_missing.csv (0 rader)
↳ sparade sat_backlinks_wrong.csv (0 rader)
↳ sparade sat_backlinks_all_refs.csv (26 rader)
↳ sparade sat_backlinks_all_osm.csv (26 rader)

❌ OSM har wikidata men pekar fel
--------------------------------
(inget)

✅ OSM back-link OK (wikidata matchar QID)
-----------------------------------------
Q134568948:
  https://www.openstreetmap.org/node/12804717601
Q135010859:
  https://www.openstreetmap.org/node/12804831701
Q135012478:
  https://www.openstreetmap.org/node/12942128203
Q135013540:
  https://www.openstreetmap.org/node/12805751201
Q135013563:
  https://www.openstreetmap.org/node/12805053401
Q135013584:
  https://www.openstreetmap.org/node/12809454601
Q135013590:
  https://www.openstreetmap.org/node/12809240302
Q135013597:
  https://www.openstreetmap.org/node/12805742901
Q135013611:
  https://www.openstree

In [7]:
import os, json, html
import requests
import pandas as pd
import folium
from folium.plugins import Fullscreen, MiniMap, MeasureControl, MousePosition
from string import Template
from datetime import datetime
import html as _html

# =========================
# KONFIG
# =========================
SAT_GEOJSON_PATH   = "SAT_full.geojson"
OUTPUT_PREFIX      = "191_WD_OSM_AED"
FETCH_FROM_WD      = True

SPARQL = r"""
SELECT ?item ?itemLabel ?coord ?img ?OSMnode ?OSMway ?OSMrel ?commonscat
WHERE {
  ?item wdt:P31 wd:Q1450682;        # AED
        wdt:P6104 wd:Q134294510.    # kopplade till SAT
  OPTIONAL { ?item wdt:P18 ?img. }
  OPTIONAL { ?item wdt:P625 ?coord }
  OPTIONAL { ?item wdt:P10689 ?OSMway }
  OPTIONAL { ?item wdt:P11693 ?OSMnode }
  OPTIONAL { ?item wdt:P402   ?OSMrel }
  OPTIONAL { ?item wdt:P373   ?commonscat }
  SERVICE wikibase:label { bd:serviceParam wikibase:language "sv,en,fr,de". }
}
ORDER BY ?itemLabel
"""

# =========================
# HELPERS
# =========================
def safe(s):
    return None if (s is None or (isinstance(s, float) and pd.isna(s))) else str(s)

def fetch_wd_to_csv(sparql: str, out_csv: str):
    url = "https://query.wikidata.org/sparql"
    headers = {
        "Accept": "application/sparql-results+json",
        "User-Agent": "SAT-map-builder/1.3 (contact: salgo60@msn.com)"
    }
    r = requests.get(url, params={"query": sparql}, headers=headers, timeout=80)
    r.raise_for_status()
    data = r.json()

    rows = []
    for b in data["results"]["bindings"]:
        gv = lambda k: b[k]["value"] if k in b else None
        rows.append({
            "item": gv("item"),
            "itemLabel": gv("itemLabel"),
            "coord": gv("coord"),
            "img": gv("img"),
            "OSMnode": gv("OSMnode"),
            "OSMway": gv("OSMway"),
            "OSMrel": gv("OSMrel"),
            "commonscat": gv("commonscat"),
        })

    df = pd.DataFrame(rows)

    # parse coord "Point(lon lat)"
    lats, lons = [], []
    for v in df["coord"].astype(str):
        lat = lon = None
        if v.startswith("Point(") and v.endswith(")"):
            parts = v[6:-1].split()
            if len(parts) == 2:
                try:
                    lon = float(parts[0]); lat = float(parts[1])
                except:
                    pass
        lats.append(lat); lons.append(lon)
    df["lat"] = lats
    df["lon"] = lons
    df.to_csv(out_csv, index=False)
    return df

def bounds_from_geojson(gj):
    minlat=minlon= 1e9
    maxlat=maxlon=-1e9
    def add(coords):
        nonlocal minlat, minlon, maxlat, maxlon
        if isinstance(coords[0], (float,int)):
            lon, lat = coords
            minlat = min(minlat, lat); maxlat = max(maxlat, lat)
            minlon = min(minlon, lon); maxlon = max(maxlon, lon)
        else:
            for c in coords:
                add(c)
    for feat in gj.get("features", []):
        geom = feat.get("geometry", {})
        add(geom.get("coordinates", []))
    if minlat == 1e9:
        return (59.3, 18.0, 59.6, 19.2)
    return (minlat, minlon, maxlat, maxlon)

# Hämta SHR-referenser via Overpass
def fetch_shr_refs(bbox):
    query = f"""
    [out:json][timeout:60];
    node
      ["ref:hjartstartarregistret.se"]
      ({bbox[0]},{bbox[1]},{bbox[2]},{bbox[3]});
    out tags;
    """
    r = requests.get("https://overpass-api.de/api/interpreter", params={"data": query}, timeout=80)
    r.raise_for_status()
    data = r.json()
    refs = {}
    for el in data.get("elements", []):
        osm_id = str(el["id"])
        ref = el["tags"].get("ref:hjartstartarregistret.se")
        if ref:
            refs[osm_id] = ref
    return refs

# =========================
# ABOUT BOX
# =========================
def add_about_box(
    m,
    issue_number: int,
    map_name: str,
    created_date: str | None = None,
    repo: str = "salgo60/Stockholm_Archipelago_Trail",
    collapsed: bool = False,
):
    if created_date is None:
        created_date = datetime.now().strftime("%Y-%m-%d %H:%M")
    map_dom_id = m.get_name()
    box_id     = f"sat-about-{map_dom_id}"
    header_id  = f"{box_id}-hdr"
    issue_url  = f"https://github.com/{repo}/issues/{issue_number}"

    links = [
        ("SAT Dashboard", "https://raw.githack.com/salgo60/Stockholm_Archipelago_Trail/main/notebook/output/SAT_ALL_IN_ONE_142_3_dashboard_latest.html"),
        ("Project repo issues", "https://github.com/salgo60/Stockholm_Archipelago_Trail/issues?q=is%3Aissue"),
        ("Trail on OSM (rel 19012437)", "https://www.openstreetmap.org/relation/19012437"),
        ("Trail on Wikicommons", "https://commons.wikimedia.org/wiki/Category:Stockholm_Archipelago_Trail"),
        ("Official page", "https://stockholmarchipelagotrail.com/"),
        ("Unofficial FB group", "https://www.facebook.com/groups/2875020699552247"),
        ("Visit Sweden", "https://traveltrade.visitsweden.com/plan/news-sweden/Stockholm-Archipelago-Trail/"),
    ]
    links_html = "".join(
        f'<div><a href="{_html.escape(u)}" target="_blank">🔗 {_html.escape(t)}</a></div>'
        for t, u in links
    )
    tpl = Template(r"""
<style>
  .sat-about { position: fixed; z-index: 10000; background: rgba(255,255,255,0.97);
    border: 2px solid #666; border-radius: 10px; box-shadow: 0 2px 6px rgba(0,0,0,0.25);
    font: 12px/1.35 system-ui, sans-serif; pointer-events: auto;
    min-width: 240px; max-width: 320px; }
  .sat-about-header { cursor: pointer; padding: 8px 10px; font-weight: 700;
    display: flex; align-items: center; gap: 6px; user-select: none;
    background: rgba(248,248,248,.9); border-bottom: 1px solid #e5e7eb; }
  .sat-about-body { padding: 8px 10px 10px 10px; }
  .sat-about-collapsed .sat-about-body { display: none; }
  .sat-chevron { margin-left: auto; transition: transform .15s ease-in-out; }
  .sat-about-collapsed .sat-chevron { transform: rotate(-90deg); }
</style>

<div id="$box_id" class="sat-about">
  <div id="$header_id" class="sat-about-header">
    <span>ℹ️ About</span><span class="sat-chevron">▸</span>
  </div>
  <div class="sat-about-body">
    <div style="font-weight:700;margin-bottom:4px;">Stockholm Archipelago Trail Map</div>
    <div>Issue: <a href="$issue_url" target="_blank">#$issue_number</a>&nbsp;&nbsp; Map: $map_name</div>
    <div>Created: $created_date</div>
    <div class="sat-links">$links_html</div>
  </div>
</div>
""")
    html_snippet = tpl.substitute(
        box_id=box_id, header_id=header_id,
        issue_url=issue_url, issue_number=str(issue_number),
        map_name=_html.escape(map_name), created_date=_html.escape(created_date),
        links_html=links_html, map_dom_id=map_dom_id
    )
    m.get_root().html.add_child(folium.Element(html_snippet))

# =========================
# 1) Fetch AED data
# =========================
if FETCH_FROM_WD:
    print("Hämtar AED från Wikidata…")
    df_aed = fetch_wd_to_csv(SPARQL, "aed_wd.csv")
else:
    df_aed = pd.read_csv("aed_wd.csv")

print(f"AED-punkter: {len(df_aed)}")

# =========================
# 2) Read SAT_full.geojson
# =========================
with open(SAT_GEOJSON_PATH, "r", encoding="utf-8") as f:
    sat_geo = json.load(f)

# =========================
# 3) Init map
# =========================
minlat, minlon, maxlat, maxlon = bounds_from_geojson(sat_geo)
center = [(minlat+maxlat)/2, (minlon+maxlon)/2]

m = folium.Map(location=center, zoom_start=10, tiles="CartoDB positron", control_scale=True)
Fullscreen().add_to(m)
MiniMap(toggle_display=True, position="bottomleft").add_to(m)
MeasureControl(position='topleft', primary_length_unit='kilometers').add_to(m)
MousePosition(position='bottomright', separator=' | ', num_digits=5, prefix='Lat/Lon:').add_to(m)

# Trail overlay
folium.GeoJson(
    sat_geo, name="SAT_full",
    style_function=lambda feat: {"color":"#e4572e","weight":5,"opacity":0.9}
).add_to(folium.FeatureGroup(name="SAT spår", show=True).add_to(m))

# =========================
# 4) Fetch SHR refs via Overpass
# =========================
bbox = (minlat, minlon, maxlat, maxlon)
shr_refs = fetch_shr_refs(bbox)
print(f"Hittade {len(shr_refs)} SHR-referenser i området.")

# =========================
# 5) Add AED points
# =========================
aed_fg = folium.FeatureGroup(name="AED", show=True)

for _, row in df_aed.iterrows():
    lat, lon = row.get("lat"), row.get("lon")
    if pd.isna(lat) or pd.isna(lon): continue
    
    itemLabel = safe(row.get("itemLabel")) or "AED"
    img       = safe(row.get("img"))
    osmnode   = safe(row.get("OSMnode"))
    osmway    = safe(row.get("OSMway"))
    osmrel    = safe(row.get("OSMrel"))
    wd_url    = safe(row.get("item"))
    commonscat = safe(row.get("commonscat"))
    

    links = []
    if osmnode:
        links.append(f"🗺️ <a href='https://www.openstreetmap.org/node/{osmnode}'>OSM node</a>")
        links.append(f"🌍 <a href='https://openaedmap.org/#node/{osmnode}'>OpenAEDMap</a>")
        if osmnode in shr_refs:
            links.append(f"❤️ <a href='https://www.hjartstartarregistret.se/#/{shr_refs[osmnode]}'>Hjärtstartarregistret</a>")
    if osmway:
        links.append(f"🗺️ <a href='https://www.openstreetmap.org/way/{osmway}'>OSM way</a>")
        links.append(f"🌍 <a href='https://openaedmap.org/#way/{osmway}'>OpenAEDMap</a>")
    if osmrel:
        links.append(f"🗺️ <a href='https://www.openstreetmap.org/relation/{osmrel}'>OSM rel</a>")
        links.append(f"🌍 <a href='https://openaedmap.org/#relation/{osmrel}'>OpenAEDMap</a>")
    if wd_url:
        links.append(f"🧠 <a href='{html.escape(wd_url)}' target='_blank'>Wikidata</a>")
    if commonscat:
        links.append(f"📷 <a href='https://commons.wikimedia.org/wiki/Category:{html.escape(commonscat)}' target='_blank'>Wikimedia Commons</a>")

    links_html = "<ul style='list-style:none; padding:0; margin:6px 0 0 0; font-size:13px;'>"
    for l in links: 
        links_html += f"<li>{l}</li>"
        links_html += "</ul>"

        img_html = f"<div><img src='{img}' style='max-width:280px; border-radius:8px;'></div>" if img else ""

        popup = f"""
        <div style="font-family:system-ui; max-width:330px;">
          <div style="font-weight:600; font-size:15px;">{html.escape(itemLabel)}</div>
          <div style="font-size:12px; opacity:0.7;">❤️ Hjärtstartare (SAT)</div>
          {img_html}
          {links_html}
        </div>
        """

    folium.Marker(
        [float(lat), float(lon)],
        icon=folium.Icon(icon="heartbeat", color="red"),
        popup=folium.Popup(popup, max_width=380)
    ).add_to(aed_fg)

aed_fg.add_to(m)
folium.LayerControl(collapsed=False).add_to(m)

add_about_box(m, issue_number=191, map_name="SAT Hjärtstartare längs leden")

# =========================
# 6) Save
# =========================
os.makedirs("output", exist_ok=True)
ts = datetime.now().strftime("%Y%m%d_%H%M%S")
out_ts = os.path.join("output", f"{OUTPUT_PREFIX}_{ts}.html")
out_latest = os.path.join("output", f"{OUTPUT_PREFIX}_latest.html")
m.save(out_ts)
m.save(out_latest)

print("Klar:")
print("  ", out_ts)
print("  ", out_latest)




Hämtar AED från Wikidata…
AED-punkter: 26
Hittade 52 SHR-referenser i området.
Klar:
   output/191_WD_OSM_AED_20250929_072431.html
   output/191_WD_OSM_AED_latest.html


In [5]:
df_aed


,item,itemLabel,coord,img,OSMnode,OSMway,OSMrel,commonscat,lat,lon
0,http://www.wikidata.org/entity/Q135900661,Arholma Intresseförening Ensfyrens väg hjärtst...,Point(19.10587 59.84042),http://commons.wikimedia.org/wiki/Special:File...,13069095802,None,None,Arholma Intresseförening Ensfyrens väg AED,59.840420,19.105870
1,http://www.wikidata.org/entity/Q135900531,Arholma handel hjärtstartare,Point(19.10793 59.85117),http://commons.wikimedia.org/wiki/Special:File...,13068560201,None,None,Arholma handel AED 13068560201,59.851170,19.107930
2,http://www.wikidata.org/entity/Q135025121,Finnhamn Ragnars kiosk hjärtstartare,Point(18.812762 59.479669),http://commons.wikimedia.org/wiki/Special:File...,12836972901,None,None,Finnhamn Ragnars kiosk AED,59.479669,18.812762
3,http://www.wikidata.org/entity/Q135034569,Finnhamns lanthandel hjärtstartare,Point(18.824314 59.482756),http://commons.wikimedia.org/wiki/Special:File...,6535493385,None,None,Finnhamns lanthandel AED,59.482756,18.824314
4,http://www.wikidata.org/entity/Q135969641,Grinda värdshus hjärtstartare,Point(18.558746 59.413522),http://commons.wikimedia.org/wiki/Special:File...,13096789602,None,None,Grinda AED,59.413522,18.558746
5,http://www.wikidata.org/entity/Q135041681,Gymnastiksalen Köpmanholms skola AED,Point(18.930951 59.65485),http://commons.wikimedia.org/wiki/Special:File...,12811090801,None,None,Gymnastiksalen Köpmanholms skola AED,59.654850,18.930951
6,http://www.wikidata.org/entity/Q135010859,Hjärtstarare västra hamnen Landsort,Point(17.865298 58.744498),http://commons.wikimedia.org/wiki/Special:File...,12804831701,None,None,Sjöfartsverket gaveln AED,58.744498,17.865298
7,http://www.wikidata.org/entity/Q135041722,Hysängen hjärtstartare,Point(18.846285 59.616467),http://commons.wikimedia.org/wiki/Special:File...,12811532701,None,None,Hysängen AED,59.616467,18.846285
8,http://www.wikidata.org/entity/Q135034710,Idholmens gårdsbutik hjärtstartare,Point(18.815508 59.483222),http://commons.wikimedia.org/wiki/Special:File...,12081567274,None,None,Idholmens gårdsbutik hjärtstartare,59.483222,18.815508
9,http://www.wikidata.org/entity/Q135041758,Kungshamns Samfällighet Hästskovägen hjärtstar...,Point(18.883756 59.636341),http://commons.wikimedia.org/wiki/Special:File...,12811267201,None,None,Kungshamns Samfällighet Hästskovägen AED,59.636341,18.883756
